In [2]:
from transformers import GPT2ForSequenceClassification


model = GPT2ForSequenceClassification.from_pretrained("Dave12121/chat3Fsentiment", num_labels=3)
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialogRPT-updown")

In [3]:
from datasets import load_dataset

sentences_75 = load_dataset("financial_phrasebank", "sentences_75agree")


In [4]:
import torch
import tensorflow as tf
predLabels = []
refLabels = []
i = 1
for sample in sentences_75["train"]:
    refLabels.append(sample['label'])
    res = tokenizer(sample["sentence"], return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        result = model(**res).logits
    predicted_class_id = int(tf.math.argmax(result, axis=-1)[0])
    predLabels.append(predicted_class_id)
    i += 1
    if (i % 100) == 0:
        print(f"Progress: {i/len(sentences_75['train'])}")

Progress: 0.028960324355632783
Progress: 0.057920648711265565
Progress: 0.08688097306689835
Progress: 0.11584129742253113
Progress: 0.1448016217781639
Progress: 0.1737619461337967
Progress: 0.20272227048942948
Progress: 0.23168259484506226
Progress: 0.26064291920069504
Progress: 0.2896032435563278
Progress: 0.3185635679119606
Progress: 0.3475238922675934
Progress: 0.3764842166232262
Progress: 0.40544454097885896
Progress: 0.43440486533449174
Progress: 0.4633651896901245
Progress: 0.4923255140457573
Progress: 0.5212858384013901
Progress: 0.5502461627570229
Progress: 0.5792064871126557
Progress: 0.6081668114682884
Progress: 0.6371271358239212
Progress: 0.666087460179554
Progress: 0.6950477845351868
Progress: 0.7240081088908196
Progress: 0.7529684332464524
Progress: 0.7819287576020851
Progress: 0.8108890819577179
Progress: 0.8398494063133507
Progress: 0.8688097306689835
Progress: 0.8977700550246163
Progress: 0.926730379380249
Progress: 0.9556907037358818
Progress: 0.9846510280915146


In [5]:
import evaluate
#1 neutral 2 positive

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

print(f1.compute(predictions = predLabels, references = refLabels, average = 'weighted'))
print(f1.compute(predictions = predLabels, references = refLabels, average = 'micro'))
print(f1.compute(predictions = predLabels, references = refLabels, average = 'macro'))
print(f1.compute(predictions = predLabels, references = refLabels, average = None))

{'f1': 0.922618970347453}
{'f1': 0.9238343469446858}
{'f1': 0.9046468079340619}
{'f1': array([0.88834951, 0.94789307, 0.87769784])}
